We'll continue by defining lists of reigons to be excluded fro the analysis:

* telomeres
* centromeres
* mm
* well-know CNVs

It's possible that one (or more) of these regions are actually significant in our analysis, but for now let's be conservative and remove them as they have been shown to be somehow problematic in the literature and the PennCNV homepage.

I just prepared the files using a combination of Excel and the command line, following most of the PennCNV guidelines.

In [8]:
cd ~/data/cnv/penncnv

cat centro_region_hg19 imm_region_hg19 tel_region_hg19 wellknown_region_hg19 > bad_regions.list

cd results
module load plink/1.07

plink --cnv-list all.cnv --cnv-make-map --noweb --out all
# CNV burden in each participant (all.cnv.indiv)
plink --map all.cnv.map --fam ../simplex.ped --cnv-list all.cnv --out all \
    --noweb --1 --cnv-check-no-overlap


@----------------------------------------------------------@
|        PLINK!       |     v1.07      |   10/Aug/2009     |
|----------------------------------------------------------|
|  (C) 2009 Shaun Purcell, GNU General Public License, v2  |
|----------------------------------------------------------|
|  For documentation, citation & bug-report instructions:  |
|        http://pngu.mgh.harvard.edu/purcell/plink/        |
@----------------------------------------------------------@

Skipping web check... [ --noweb ] 
Writing this text to log file [ all.log ]
Analysis started: Tue Dec 19 10:24:34 2017

Options in effect:
	--cnv-list all.cnv
	--cnv-make-map
	--noweb
	--out all


Reading segment list (CNVs) from [ all.cnv ]
Writing new MAP file to [ all.cnv.map ]
Wrote 3733 unique positions to file

Analysis finished: Tue Dec 19 10:24:34 2017


@----------------------------------------------------------@
|        PLINK!       |     v1.07      |   10/Aug/2009     |
|---------------------

In [9]:
# deletions only
plink --map all.cnv.map --fam ../simplex.ped --cnv-list all.cnv --noweb --1 \
    --cnv-del --out all_dels
# remove bad regions
plink --map all.cnv.map --fam ../simplex.ped --cnv-list all.cnv --noweb --1 \
    --cnv-exclude ../bad_regions.list --cnv-overlap .5 --out all_clean


@----------------------------------------------------------@
|        PLINK!       |     v1.07      |   10/Aug/2009     |
|----------------------------------------------------------|
|  (C) 2009 Shaun Purcell, GNU General Public License, v2  |
|----------------------------------------------------------|
|  For documentation, citation & bug-report instructions:  |
|        http://pngu.mgh.harvard.edu/purcell/plink/        |
@----------------------------------------------------------@

Skipping web check... [ --noweb ] 
Writing this text to log file [ all_dels.log ]
Analysis started: Tue Dec 19 10:24:46 2017

Options in effect:
	--map all.cnv.map
	--fam ../simplex.ped
	--cnv-list all.cnv
	--noweb
	--1
	--cnv-del
	--out all_dels

Reading marker information from [ all.cnv.map ]
3733 (of 3733) markers to be included from [ all.cnv.map ]
Reading individual information from [ ../simplex.ped ]
Reading pedigree information from [ ../simplex.ped ] 
99 individuals read from [ ../simplex.ped ] 
9

For the gene intersection, it might not make much sense to clean before hand, as the gene regions might be clean enough? Also, the overlap threshold cannot be applied separately, and we want any overlap with the gene set (not fractional):

In [16]:
# gene set
plink --map all.cnv.map --fam ../simplex.ped --cnv-list all.cnv --noweb --1 \
    --cnv-intersect ../glist-hg19 --cnv-subset ../genes.txt --out all_gene \
    --cnv-verbose-report-regions


@----------------------------------------------------------@
|        PLINK!       |     v1.07      |   10/Aug/2009     |
|----------------------------------------------------------|
|  (C) 2009 Shaun Purcell, GNU General Public License, v2  |
|----------------------------------------------------------|
|  For documentation, citation & bug-report instructions:  |
|        http://pngu.mgh.harvard.edu/purcell/plink/        |
@----------------------------------------------------------@

Skipping web check... [ --noweb ] 
Writing this text to log file [ all_gene.log ]
Analysis started: Tue Dec 19 10:40:53 2017

Options in effect:
	--map all.cnv.map
	--fam ../simplex.ped
	--cnv-list all.cnv
	--noweb
	--1
	--cnv-intersect ../glist-hg19
	--cnv-subset ../genes.txt
	--out all_gene
	--cnv-verbose-report-regions

Reading marker information from [ all.cnv.map ]
3733 (of 3733) markers to be included from [ all.cnv.map ]
Reading individual information from [ ../simplex.ped ]
Reading pedigree inform

OK, so the examples above should take care of all the possible transformations we need. 

These are different permutations to analyze:

 * clean / unclean
 * all / denovo / inherited
 * raw / merged .2 / merged .5
 * duplication / deletion
 * all / gene set intersect / brain intersect
 * SNP number threshold
 * CNV length threshold

In [21]:
cd ~/data/cnv/penncnv

cat centro_region_hg19 imm_region_hg19 tel_region_hg19 wellknown_region_hg19 > bad_regions.list

cd results
module load plink/1.07

for cnvtype in all denovo inherited; do
    for comb in '' '_comb.2' '_comb.5'; do
        fname=${cnvtype}.jointcnv${comb}
        cnvname=${cnvtype}${comb}.cnv
        ../penncnv_to_plink.pl -i $fname -o $cnvname -f ../simplex.ped;
        plink --cnv-list $cnvname --cnv-make-map --noweb --out ${cnvtype}${comb};
        # whole burden
        plink --map ${cnvname}.map --fam ../simplex.ped --cnv-list $cnvname \
            --noweb --1 --cnv-check-no-overlap --out ${cnvtype}${comb}_burden;
        # gene sets
        plink --map ${cnvname}.map --fam ../simplex.ped --cnv-list $cnvname \
            --noweb --1 --cnv-intersect ../glist-hg19 --cnv-verbose-report-regions \
            --cnv-subset ../genes.txt --out ${cnvtype}${comb}_genes;
        plink --map ${cnvname}.map --fam ../simplex.ped --cnv-list $cnvname \
            --noweb --1 --cnv-intersect ../glist-hg19 --cnv-verbose-report-regions \
            --cnv-subset ../hg19_allenBrainGene_trimmed.txt \
            --out ${cnvtype}${comb}_brainGenes;
        # subtypes only
        for sub in del dup; do
            plink --map ${cnvname}.map --fam ../simplex.ped --cnv-list $cnvname \
            --noweb --1 --cnv-${sub} --out ${cnvtype}${comb}_${sub}Burden;
            # gene sets
            plink --map ${cnvname}.map --fam ../simplex.ped --cnv-list $cnvname \
                --noweb --1 --cnv-intersect ../glist-hg19 --cnv-verbose-report-regions \
                --cnv-subset ../genes.txt --cnv-${sub} \
                --out ${cnvtype}${comb}_${sub}Genes;
            plink --map ${cnvname}.map --fam ../simplex.ped --cnv-list $cnvname \
                --noweb --1 --cnv-intersect ../glist-hg19 --cnv-verbose-report-regions \
                --cnv-subset ../hg19_allenBrainGene_trimmed.txt --cnv-${sub} \
                --out ${cnvtype}${comb}_${sub}BrainGenes;
        done;
    done;
done


@----------------------------------------------------------@
|        PLINK!       |     v1.07      |   10/Aug/2009     |
|----------------------------------------------------------|
|  (C) 2009 Shaun Purcell, GNU General Public License, v2  |
|----------------------------------------------------------|
|  For documentation, citation & bug-report instructions:  |
|        http://pngu.mgh.harvard.edu/purcell/plink/        |
@----------------------------------------------------------@

Skipping web check... [ --noweb ] 
Writing this text to log file [ all.log ]
Analysis started: Tue Dec 19 11:26:11 2017

Options in effect:
	--cnv-list all.cnv
	--cnv-make-map
	--noweb
	--out all


Reading segment list (CNVs) from [ all.cnv ]
Writing new MAP file to [ all.cnv.map ]
Wrote 3733 unique positions to file

Analysis finished: Tue Dec 19 11:26:11 2017


@----------------------------------------------------------@
|        PLINK!       |     v1.07      |   10/Aug/2009     |
|---------------------

|  (C) 2009 Shaun Purcell, GNU General Public License, v2  |
|----------------------------------------------------------|
|  For documentation, citation & bug-report instructions:  |
|        http://pngu.mgh.harvard.edu/purcell/plink/        |
@----------------------------------------------------------@

Skipping web check... [ --noweb ] 
Writing this text to log file [ all_delGenes.log ]
Analysis started: Tue Dec 19 11:26:13 2017

Options in effect:
	--map all.cnv.map
	--fam ../simplex.ped
	--cnv-list all.cnv
	--noweb
	--1
	--cnv-intersect ../glist-hg19
	--cnv-verbose-report-regions
	--cnv-subset ../genes.txt
	--cnv-del
	--out all_delGenes

Reading marker information from [ all.cnv.map ]
3733 (of 3733) markers to be included from [ all.cnv.map ]
Reading individual information from [ ../simplex.ped ]
Reading pedigree information from [ ../simplex.ped ] 
99 individuals read from [ ../simplex.ped ] 
99 individuals with nonmissing phenotypes
Assuming a disease phenotype (0=unaff, 1=aff, o


Reading marker information from [ all.cnv.map ]
3733 (of 3733) markers to be included from [ all.cnv.map ]
Reading individual information from [ ../simplex.ped ]
Reading pedigree information from [ ../simplex.ped ] 
99 individuals read from [ ../simplex.ped ] 
99 individuals with nonmissing phenotypes
Assuming a disease phenotype (0=unaff, 1=aff, other=miss)
21 cases, 78 controls and 0 missing
56 males, 43 females, and 0 of unspecified sex
Reading intersection subset list from [ ../hg19_allenBrainGene_trimmed.txt ]
Looking for subset of 606 ranges
Reading CNV intersection list from [ ../glist-hg19 ]
Read 577 ranges to intersect with CNV list
Could not find 29 ranges
Writing this list to [ all_dupBrainGenes.notfound ] 

Reading segment list (CNVs) from [ all.cnv ]
Writing back list to intersected regions to [ all_dupBrainGenes.reg ]
1882 mapped to a person, of which 466 passed filters
22 intersected with one or more specified region
22 of 1882 mapped as valid segments
 CopyN Case/Contr

|        PLINK!       |     v1.07      |   10/Aug/2009     |
|----------------------------------------------------------|
|  (C) 2009 Shaun Purcell, GNU General Public License, v2  |
|----------------------------------------------------------|
|  For documentation, citation & bug-report instructions:  |
|        http://pngu.mgh.harvard.edu/purcell/plink/        |
@----------------------------------------------------------@

Skipping web check... [ --noweb ] 
Writing this text to log file [ all_comb.2_delBurden.log ]
Analysis started: Tue Dec 19 11:26:19 2017

Options in effect:
	--map all_comb.2.cnv.map
	--fam ../simplex.ped
	--cnv-list all_comb.2.cnv
	--noweb
	--1
	--cnv-del
	--out all_comb.2_delBurden

** For gPLINK compatibility, do not use '.' in --out **
Reading marker information from [ all_comb.2.cnv.map ]
3652 (of 3652) markers to be included from [ all_comb.2.cnv.map ]
Reading individual information from [ ../simplex.ped ]
Reading pedigree information from [ ../simplex.ped ] 


	--map all_comb.2.cnv.map
	--fam ../simplex.ped
	--cnv-list all_comb.2.cnv
	--noweb
	--1
	--cnv-intersect ../glist-hg19
	--cnv-verbose-report-regions
	--cnv-subset ../genes.txt
	--cnv-dup
	--out all_comb.2_dupGenes

** For gPLINK compatibility, do not use '.' in --out **
Reading marker information from [ all_comb.2.cnv.map ]
3652 (of 3652) markers to be included from [ all_comb.2.cnv.map ]
Reading individual information from [ ../simplex.ped ]
Reading pedigree information from [ ../simplex.ped ] 
99 individuals read from [ ../simplex.ped ] 
99 individuals with nonmissing phenotypes
Assuming a disease phenotype (0=unaff, 1=aff, other=miss)
21 cases, 78 controls and 0 missing
56 males, 43 females, and 0 of unspecified sex
Reading intersection subset list from [ ../genes.txt ]
Looking for subset of 25303 ranges
Reading CNV intersection list from [ ../glist-hg19 ]
Read 25646 ranges to intersect with CNV list

Reading segment list (CNVs) from [ all_comb.2.cnv ]
Writing back list to intersec

...

Reading individual information from [ ../simplex.ped ]
Reading pedigree information from [ ../simplex.ped ] 
99 individuals read from [ ../simplex.ped ] 
99 individuals with nonmissing phenotypes
Assuming a disease phenotype (0=unaff, 1=aff, other=miss)
21 cases, 78 controls and 0 missing
56 males, 43 females, and 0 of unspecified sex

Reading segment list (CNVs) from [ inherited_comb.2.cnv ]
No overlapping samples found
504 mapped to a person, of which 504 passed filters
504 of 504 mapped as valid segments
 CopyN Case/Control
     0        1 / 0
     1    100 / 174
     3     42 / 186
     4        1 / 0

Writing per-individual summary to [ inherited_comb.2_burden.cnv.indiv ]
Writing positional summary to [ inherited_comb.2_burden.cnv.summary ]

Analysis finished: Tue Dec 19 11:26:43 2017


@----------------------------------------------------------@
|        PLINK!       |     v1.07      |   10/Aug/2009     |
|----------------------------------------------------------|
|  (C) 2009 Sh

Assuming a disease phenotype (0=unaff, 1=aff, other=miss)
21 cases, 78 controls and 0 missing
56 males, 43 females, and 0 of unspecified sex
Reading intersection subset list from [ ../genes.txt ]
Looking for subset of 25303 ranges
Reading CNV intersection list from [ ../glist-hg19 ]
Read 25646 ranges to intersect with CNV list

Reading segment list (CNVs) from [ inherited_comb.2.cnv ]
Writing back list to intersected regions to [ inherited_comb.2_delGenes.reg ]
504 mapped to a person, of which 275 passed filters
273 intersected with one or more specified region
273 of 504 mapped as valid segments
 CopyN Case/Control
     0        1 / 0
     1     99 / 173

Writing per-individual summary to [ inherited_comb.2_delGenes.cnv.indiv ]
Writing positional summary to [ inherited_comb.2_delGenes.cnv.summary ]

Analysis finished: Tue Dec 19 11:26:45 2017


@----------------------------------------------------------@
|        PLINK!       |     v1.07      |   10/Aug/2009     |
|-------------------

99 individuals with nonmissing phenotypes
Assuming a disease phenotype (0=unaff, 1=aff, other=miss)
21 cases, 78 controls and 0 missing
56 males, 43 females, and 0 of unspecified sex
Reading intersection subset list from [ ../hg19_allenBrainGene_trimmed.txt ]
Looking for subset of 606 ranges
Reading CNV intersection list from [ ../glist-hg19 ]
Read 577 ranges to intersect with CNV list
Could not find 29 ranges
Writing this list to [ inherited_comb.2_dupBrainGenes.notfound ] 

Reading segment list (CNVs) from [ inherited_comb.2.cnv ]
Writing back list to intersected regions to [ inherited_comb.2_dupBrainGenes.reg ]
504 mapped to a person, of which 229 passed filters
14 intersected with one or more specified region
14 of 504 mapped as valid segments
 CopyN Case/Control
     3        5 / 9

Writing per-individual summary to [ inherited_comb.2_dupBrainGenes.cnv.indiv ]
Writing positional summary to [ inherited_comb.2_dupBrainGenes.cnv.summary ]

Analysis finished: Tue Dec 19 11:26:46 2017


|        PLINK!       |     v1.07      |   10/Aug/2009     |
|----------------------------------------------------------|
|  (C) 2009 Shaun Purcell, GNU General Public License, v2  |
|----------------------------------------------------------|
|  For documentation, citation & bug-report instructions:  |
|        http://pngu.mgh.harvard.edu/purcell/plink/        |
@----------------------------------------------------------@

Skipping web check... [ --noweb ] 
Writing this text to log file [ inherited_comb.5_delBurden.log ]
Analysis started: Tue Dec 19 11:26:47 2017

Options in effect:
	--map inherited_comb.5.cnv.map
	--fam ../simplex.ped
	--cnv-list inherited_comb.5.cnv
	--noweb
	--1
	--cnv-del
	--out inherited_comb.5_delBurden

** For gPLINK compatibility, do not use '.' in --out **
Reading marker information from [ inherited_comb.5.cnv.map ]
1017 (of 1017) markers to be included from [ inherited_comb.5.cnv.map ]
Reading individual information from [ ../simplex.ped ]
Reading pedigree i

@----------------------------------------------------------@

Skipping web check... [ --noweb ] 
Writing this text to log file [ inherited_comb.5_dupGenes.log ]
Analysis started: Tue Dec 19 11:26:47 2017

Options in effect:
	--map inherited_comb.5.cnv.map
	--fam ../simplex.ped
	--cnv-list inherited_comb.5.cnv
	--noweb
	--1
	--cnv-intersect ../glist-hg19
	--cnv-verbose-report-regions
	--cnv-subset ../genes.txt
	--cnv-dup
	--out inherited_comb.5_dupGenes

** For gPLINK compatibility, do not use '.' in --out **
Reading marker information from [ inherited_comb.5.cnv.map ]
1017 (of 1017) markers to be included from [ inherited_comb.5.cnv.map ]
Reading individual information from [ ../simplex.ped ]
Reading pedigree information from [ ../simplex.ped ] 
99 individuals read from [ ../simplex.ped ] 
99 individuals with nonmissing phenotypes
Assuming a disease phenotype (0=unaff, 1=aff, other=miss)
21 cases, 78 controls and 0 missing
56 males, 43 females, and 0 of unspecified sex
Reading intersec

Before we introduce the cleaning variables to it, let's do some plotting. But we'll need a partner Python notebook for it.

# Clean up

In [23]:
cd ~/data/cnv/penncnv

cat centro_region_hg19 imm_region_hg19 tel_region_hg19 wellknown_region_hg19 > bad_regions.list

cd results
module load plink/1.07

for cnvtype in all denovo inherited; do
    for comb in '' '_comb.2' '_comb.5'; do
        cnvname=${cnvtype}${comb}.cnv
        
        # remove bad regions
        plink --map ${cnvname}.map --fam ../simplex.ped --cnv-list $cnvname \
            --noweb --1 --cnv-exclude ../bad_regions.list --cnv-overlap .5 \
            --cnv-write --out ${cnvtype}${comb}_clean
        plink --cnv-list ${cnvtype}${comb}_clean.cnv --cnv-make-map --noweb --1 \
            --out ${cnvtype}${comb}_clean
    done;
done


@----------------------------------------------------------@
|        PLINK!       |     v1.07      |   10/Aug/2009     |
|----------------------------------------------------------|
|  (C) 2009 Shaun Purcell, GNU General Public License, v2  |
|----------------------------------------------------------|
|  For documentation, citation & bug-report instructions:  |
|        http://pngu.mgh.harvard.edu/purcell/plink/        |
@----------------------------------------------------------@

Skipping web check... [ --noweb ] 
Writing this text to log file [ all_clean.log ]
Analysis started: Tue Dec 19 14:14:05 2017

Options in effect:
	--map all.cnv.map
	--fam ../simplex.ped
	--cnv-list all.cnv
	--noweb
	--1
	--cnv-exclude ../bad_regions.list
	--cnv-overlap .5
	--cnv-write
	--out all_clean

Reading marker information from [ all.cnv.map ]
3733 (of 3733) markers to be included from [ all.cnv.map ]
Reading individual information from [ ../simplex.ped ]
Reading pedigree information from [ ../simp



@----------------------------------------------------------@
|        PLINK!       |     v1.07      |   10/Aug/2009     |
|----------------------------------------------------------|
|  (C) 2009 Shaun Purcell, GNU General Public License, v2  |
|----------------------------------------------------------|
|  For documentation, citation & bug-report instructions:  |
|        http://pngu.mgh.harvard.edu/purcell/plink/        |
@----------------------------------------------------------@

Skipping web check... [ --noweb ] 
Writing this text to log file [ denovo_clean.log ]
Analysis started: Tue Dec 19 14:15:28 2017

Options in effect:
	--map denovo.cnv.map
	--fam ../simplex.ped
	--cnv-list denovo.cnv
	--noweb
	--1
	--cnv-exclude ../bad_regions.list
	--cnv-overlap .5
	--cnv-write
	--out denovo_clean

Reading marker information from [ denovo.cnv.map ]
2932 (of 2932) markers to be included from [ denovo.cnv.map ]
Reading individual information from [ ../simplex.ped ]
Reading pedigree informa

Writing new MAP file to [ denovo_comb.5_clean.cnv.map ]
Wrote 239 unique positions to file

Analysis finished: Tue Dec 19 14:16:30 2017


@----------------------------------------------------------@
|        PLINK!       |     v1.07      |   10/Aug/2009     |
|----------------------------------------------------------|
|  (C) 2009 Shaun Purcell, GNU General Public License, v2  |
|----------------------------------------------------------|
|  For documentation, citation & bug-report instructions:  |
|        http://pngu.mgh.harvard.edu/purcell/plink/        |
@----------------------------------------------------------@

Skipping web check... [ --noweb ] 
Writing this text to log file [ inherited_clean.log ]
Analysis started: Tue Dec 19 14:16:30 2017

Options in effect:
	--map inherited.cnv.map
	--fam ../simplex.ped
	--cnv-list inherited.cnv
	--noweb
	--1
	--cnv-exclude ../bad_regions.list
	--cnv-overlap .5
	--cnv-write
	--out inherited_clean

Reading marker information from [ inherited.


Options in effect:
	--cnv-list inherited_comb.5_clean.cnv
	--cnv-make-map
	--noweb
	--1
	--out inherited_comb.5_clean

** For gPLINK compatibility, do not use '.' in --out **

Reading segment list (CNVs) from [ inherited_comb.5_clean.cnv ]
Writing new MAP file to [ inherited_comb.5_clean.cnv.map ]
Wrote 62 unique positions to file

Analysis finished: Tue Dec 19 14:16:55 2017



Note that the code below re-generates the .indiv files we had before, but now it also does the _clean set and the SNP and CNV thresholds.

In [24]:
cd ~/data/cnv/penncnv/results

for cnvtype in all denovo inherited; do
    for comb in '' '_comb.2' '_comb.5'; do
        for qc in '' '_clean'; do
            cnvname=${cnvtype}${comb}${qc}.cnv
            # apply CNV length threshold in clean and nonclean sets
            for kb in 0 50 100 500; do
                # whole burden
                plink --map ${cnvname}.map --fam ../simplex.ped --cnv-list $cnvname \
                    --noweb --1 --cnv-check-no-overlap --cnv-kb $kb \
                    --out ${cnvtype}${comb}${qc}_lenBT${kb}_burden;
                # gene sets
                plink --map ${cnvname}.map --fam ../simplex.ped --cnv-list $cnvname \
                    --noweb --1 --cnv-intersect ../glist-hg19 --cnv-verbose-report-regions \
                    --cnv-subset ../genes.txt --cnv-kb $kb \
                    --out ${cnvtype}${comb}${qc}_lenBT${kb}_genes;
                plink --map ${cnvname}.map --fam ../simplex.ped --cnv-list $cnvname \
                    --noweb --1 --cnv-intersect ../glist-hg19 --cnv-verbose-report-regions \
                    --cnv-subset ../hg19_allenBrainGene_trimmed.txt --cnv-kb $kb \
                    --out ${cnvtype}${comb}${qc}_lenBT${kb}_brainGenes;
                # subtypes only
                for sub in del dup; do
                    plink --map ${cnvname}.map --fam ../simplex.ped --cnv-list $cnvname \
                    --noweb --1 --cnv-${sub} --cnv-kb $kb \
                    --out ${cnvtype}${comb}${qc}_lenBT${kb}_${sub}Burden;
                    # gene sets
                    plink --map ${cnvname}.map --fam ../simplex.ped --cnv-list $cnvname \
                        --noweb --1 --cnv-intersect ../glist-hg19 --cnv-verbose-report-regions \
                        --cnv-subset ../genes.txt --cnv-${sub} --cnv-kb $kb\
                        --out ${cnvtype}${comb}${qc}_lenBT${kb}_${sub}Genes;
                    plink --map ${cnvname}.map --fam ../simplex.ped --cnv-list $cnvname \
                        --noweb --1 --cnv-intersect ../glist-hg19 --cnv-verbose-report-regions \
                        --cnv-subset ../hg19_allenBrainGene_trimmed.txt --cnv-${sub} \
                        --cnv-kb $kb \
                        --out ${cnvtype}${comb}${qc}_lenBT${kb}_${sub}BrainGenes;
                done;
            done;
        done;
    done;
done


@----------------------------------------------------------@
|        PLINK!       |     v1.07      |   10/Aug/2009     |
|----------------------------------------------------------|
|  (C) 2009 Shaun Purcell, GNU General Public License, v2  |
|----------------------------------------------------------|
|  For documentation, citation & bug-report instructions:  |
|        http://pngu.mgh.harvard.edu/purcell/plink/        |
@----------------------------------------------------------@

Skipping web check... [ --noweb ] 
Writing this text to log file [ all_lenBT0_burden.log ]
Analysis started: Tue Dec 19 14:27:07 2017

Options in effect:
	--map all.cnv.map
	--fam ../simplex.ped
	--cnv-list all.cnv
	--noweb
	--1
	--cnv-check-no-overlap
	--cnv-kb 0
	--out all_lenBT0_burden

Reading marker information from [ all.cnv.map ]
3733 (of 3733) markers to be included from [ all.cnv.map ]
Reading individual information from [ ../simplex.ped ]
Reading pedigree information from [ ../simplex.ped ] 
99 

	--cnv-del
	--cnv-kb 0
	--out all_lenBT0_delGenes

Reading marker information from [ all.cnv.map ]
3733 (of 3733) markers to be included from [ all.cnv.map ]
Reading individual information from [ ../simplex.ped ]
Reading pedigree information from [ ../simplex.ped ] 
99 individuals read from [ ../simplex.ped ] 
99 individuals with nonmissing phenotypes
Assuming a disease phenotype (0=unaff, 1=aff, other=miss)
21 cases, 78 controls and 0 missing
56 males, 43 females, and 0 of unspecified sex
Reading intersection subset list from [ ../genes.txt ]
Looking for subset of 25303 ranges
Reading CNV intersection list from [ ../glist-hg19 ]
Read 25646 ranges to intersect with CNV list

Reading segment list (CNVs) from [ all.cnv ]
Writing back list to intersected regions to [ all_lenBT0_delGenes.reg ]
1882 mapped to a person, of which 1416 passed filters
1410 intersected with one or more specified region
1410 of 1882 mapped as valid segments
 CopyN Case/Control
     0        2 / 3
     1    626 / 

Reading intersection subset list from [ ../hg19_allenBrainGene_trimmed.txt ]
Looking for subset of 606 ranges
Reading CNV intersection list from [ ../glist-hg19 ]
Read 577 ranges to intersect with CNV list
Could not find 29 ranges
Writing this list to [ all_lenBT0_dupBrainGenes.notfound ] 

Reading segment list (CNVs) from [ all.cnv ]
Writing back list to intersected regions to [ all_lenBT0_dupBrainGenes.reg ]
1882 mapped to a person, of which 466 passed filters
22 intersected with one or more specified region
22 of 1882 mapped as valid segments
 CopyN Case/Control
     3       7 / 15

Writing per-individual summary to [ all_lenBT0_dupBrainGenes.cnv.indiv ]
Writing positional summary to [ all_lenBT0_dupBrainGenes.cnv.summary ]

Analysis finished: Tue Dec 19 14:27:13 2017


@----------------------------------------------------------@
|        PLINK!       |     v1.07      |   10/Aug/2009     |
|----------------------------------------------------------|
|  (C) 2009 Shaun Purcell, GNU Ge

|----------------------------------------------------------|
|  (C) 2009 Shaun Purcell, GNU General Public License, v2  |
|----------------------------------------------------------|
|  For documentation, citation & bug-report instructions:  |
|        http://pngu.mgh.harvard.edu/purcell/plink/        |
@----------------------------------------------------------@

Skipping web check... [ --noweb ] 
Writing this text to log file [ all_lenBT50_delGenes.log ]
Analysis started: Tue Dec 19 14:27:14 2017

Options in effect:
	--map all.cnv.map
	--fam ../simplex.ped
	--cnv-list all.cnv
	--noweb
	--1
	--cnv-intersect ../glist-hg19
	--cnv-verbose-report-regions
	--cnv-subset ../genes.txt
	--cnv-del
	--cnv-kb 50
	--out all_lenBT50_delGenes

Reading marker information from [ all.cnv.map ]
3733 (of 3733) markers to be included from [ all.cnv.map ]
Reading individual information from [ ../simplex.ped ]
Reading pedigree information from [ ../simplex.ped ] 
99 individuals read from [ ../simplex.ped ] 

	--fam ../simplex.ped
	--cnv-list all.cnv
	--noweb
	--1
	--cnv-intersect ../glist-hg19
	--cnv-verbose-report-regions
	--cnv-subset ../hg19_allenBrainGene_trimmed.txt
	--cnv-dup
	--cnv-kb 50
	--out all_lenBT50_dupBrainGenes

Reading marker information from [ all.cnv.map ]
3733 (of 3733) markers to be included from [ all.cnv.map ]
Reading individual information from [ ../simplex.ped ]
Reading pedigree information from [ ../simplex.ped ] 
99 individuals read from [ ../simplex.ped ] 
99 individuals with nonmissing phenotypes
Assuming a disease phenotype (0=unaff, 1=aff, other=miss)
21 cases, 78 controls and 0 missing
56 males, 43 females, and 0 of unspecified sex
Reading intersection subset list from [ ../hg19_allenBrainGene_trimmed.txt ]
Looking for subset of 606 ranges
Reading CNV intersection list from [ ../glist-hg19 ]
Read 577 ranges to intersect with CNV list
Could not find 29 ranges
Writing this list to [ all_lenBT50_dupBrainGenes.notfound ] 

Reading segment list (CNVs) from [ all.

...

No overlapping samples found
35 mapped to a person, of which 4 passed filters
4 of 35 mapped as valid segments
 CopyN Case/Control
     1        1 / 0
     3        1 / 2

Writing per-individual summary to [ inherited_comb.5_clean_lenBT100_burden.cnv.indiv ]
Writing positional summary to [ inherited_comb.5_clean_lenBT100_burden.cnv.summary ]

Analysis finished: Tue Dec 19 14:29:10 2017


@----------------------------------------------------------@
|        PLINK!       |     v1.07      |   10/Aug/2009     |
|----------------------------------------------------------|
|  (C) 2009 Shaun Purcell, GNU General Public License, v2  |
|----------------------------------------------------------|
|  For documentation, citation & bug-report instructions:  |
|        http://pngu.mgh.harvard.edu/purcell/plink/        |
@----------------------------------------------------------@

Skipping web check... [ --noweb ] 
Writing this text to log file [ inherited_comb.5_clean_lenBT100_genes.log ]
Analysis 

Reading intersection subset list from [ ../genes.txt ]
Looking for subset of 25303 ranges
Reading CNV intersection list from [ ../glist-hg19 ]
Read 25646 ranges to intersect with CNV list

Reading segment list (CNVs) from [ inherited_comb.5_clean.cnv ]
Writing back list to intersected regions to [ inherited_comb.5_clean_lenBT100_delGenes.reg ]
35 mapped to a person, of which 1 passed filters
1 intersected with one or more specified region
1 of 35 mapped as valid segments
 CopyN Case/Control
     1        1 / 0

Writing per-individual summary to [ inherited_comb.5_clean_lenBT100_delGenes.cnv.indiv ]
Writing positional summary to [ inherited_comb.5_clean_lenBT100_delGenes.cnv.summary ]

Analysis finished: Tue Dec 19 14:29:11 2017


@----------------------------------------------------------@
|        PLINK!       |     v1.07      |   10/Aug/2009     |
|----------------------------------------------------------|
|  (C) 2009 Shaun Purcell, GNU General Public License, v2  |
|---------------

62 (of 62) markers to be included from [ inherited_comb.5_clean.cnv.map ]
Reading individual information from [ ../simplex.ped ]
Reading pedigree information from [ ../simplex.ped ] 
99 individuals read from [ ../simplex.ped ] 
99 individuals with nonmissing phenotypes
Assuming a disease phenotype (0=unaff, 1=aff, other=miss)
21 cases, 78 controls and 0 missing
56 males, 43 females, and 0 of unspecified sex
Reading intersection subset list from [ ../hg19_allenBrainGene_trimmed.txt ]
Looking for subset of 606 ranges
Reading CNV intersection list from [ ../glist-hg19 ]
Read 577 ranges to intersect with CNV list
Could not find 29 ranges
Writing this list to [ inherited_comb.5_clean_lenBT100_dupBrainGenes.notfound ] 

Reading segment list (CNVs) from [ inherited_comb.5_clean.cnv ]
Writing back list to intersected regions to [ inherited_comb.5_clean_lenBT100_dupBrainGenes.reg ]
35 mapped to a person, of which 3 passed filters
0 intersected with one or more specified region
0 of 35 mapped as

Analysis started: Tue Dec 19 14:29:12 2017

Options in effect:
	--map inherited_comb.5_clean.cnv.map
	--fam ../simplex.ped
	--cnv-list inherited_comb.5_clean.cnv
	--noweb
	--1
	--cnv-del
	--cnv-kb 500
	--out inherited_comb.5_clean_lenBT500_delBurden

** For gPLINK compatibility, do not use '.' in --out **
Reading marker information from [ inherited_comb.5_clean.cnv.map ]
62 (of 62) markers to be included from [ inherited_comb.5_clean.cnv.map ]
Reading individual information from [ ../simplex.ped ]
Reading pedigree information from [ ../simplex.ped ] 
99 individuals read from [ ../simplex.ped ] 
99 individuals with nonmissing phenotypes
Assuming a disease phenotype (0=unaff, 1=aff, other=miss)
21 cases, 78 controls and 0 missing
56 males, 43 females, and 0 of unspecified sex

Reading segment list (CNVs) from [ inherited_comb.5_clean.cnv ]
35 mapped to a person, of which 1 passed filters
1 of 35 mapped as valid segments
 CopyN Case/Control
     1        1 / 0

Writing per-individual summ


Options in effect:
	--map inherited_comb.5_clean.cnv.map
	--fam ../simplex.ped
	--cnv-list inherited_comb.5_clean.cnv
	--noweb
	--1
	--cnv-intersect ../glist-hg19
	--cnv-verbose-report-regions
	--cnv-subset ../genes.txt
	--cnv-dup
	--cnv-kb 500
	--out inherited_comb.5_clean_lenBT500_dupGenes

** For gPLINK compatibility, do not use '.' in --out **
Reading marker information from [ inherited_comb.5_clean.cnv.map ]
62 (of 62) markers to be included from [ inherited_comb.5_clean.cnv.map ]
Reading individual information from [ ../simplex.ped ]
Reading pedigree information from [ ../simplex.ped ] 
99 individuals read from [ ../simplex.ped ] 
99 individuals with nonmissing phenotypes
Assuming a disease phenotype (0=unaff, 1=aff, other=miss)
21 cases, 78 controls and 0 missing
56 males, 43 females, and 0 of unspecified sex
Reading intersection subset list from [ ../genes.txt ]
Looking for subset of 25303 ranges
Reading CNV intersection list from [ ../glist-hg19 ]
Read 25646 ranges to interse

In [26]:
cd ~/data/cnv/penncnv/results

for cnvtype in all denovo inherited; do
    for comb in '' '_comb.2' '_comb.5'; do
        for qc in '' '_clean'; do
            cnvname=${cnvtype}${comb}${qc}.cnv
            # apply CNV length threshold in clean and nonclean sets
            for nsnp in 0 5 10 15; do
                # whole burden
                plink --map ${cnvname}.map --fam ../simplex.ped --cnv-list $cnvname \
                    --noweb --1 --cnv-check-no-overlap --cnv-sites $nsnp \
                    --out ${cnvtype}${comb}${qc}_sitesBT${nsnp}_burden;
                # gene sets
                plink --map ${cnvname}.map --fam ../simplex.ped --cnv-list $cnvname \
                    --noweb --1 --cnv-intersect ../glist-hg19 --cnv-verbose-report-regions \
                    --cnv-subset ../genes.txt --cnv-sites $nsnp \
                    --out ${cnvtype}${comb}${qc}_sitesBT${nsnp}_genes;
                plink --map ${cnvname}.map --fam ../simplex.ped --cnv-list $cnvname \
                    --noweb --1 --cnv-intersect ../glist-hg19 --cnv-verbose-report-regions \
                    --cnv-subset ../hg19_allenBrainGene_trimmed.txt --cnv-sites $nsnp \
                    --out ${cnvtype}${comb}${qc}_sitesBT${nsnp}_brainGenes;
                # subtypes only
                for sub in del dup; do
                    plink --map ${cnvname}.map --fam ../simplex.ped --cnv-list $cnvname \
                    --noweb --1 --cnv-${sub} --cnv-sites $nsnp \
                    --out ${cnvtype}${comb}${qc}_sitesBT${nsnp}_${sub}Burden;
                    # gene sets
                    plink --map ${cnvname}.map --fam ../simplex.ped --cnv-list $cnvname \
                        --noweb --1 --cnv-intersect ../glist-hg19 --cnv-verbose-report-regions \
                        --cnv-subset ../genes.txt --cnv-${sub} --cnv-sites $nsnp\
                        --out ${cnvtype}${comb}${qc}_sitesBT${nsnp}_${sub}Genes;
                    plink --map ${cnvname}.map --fam ../simplex.ped --cnv-list $cnvname \
                        --noweb --1 --cnv-intersect ../glist-hg19 --cnv-verbose-report-regions \
                        --cnv-subset ../hg19_allenBrainGene_trimmed.txt --cnv-${sub} \
                        --cnv-sites $nsnp \
                        --out ${cnvtype}${comb}${qc}_sitesBT${nsnp}_${sub}BrainGenes;
                done;
            done;
        done;
    done;
done


@----------------------------------------------------------@
|        PLINK!       |     v1.07      |   10/Aug/2009     |
|----------------------------------------------------------|
|  (C) 2009 Shaun Purcell, GNU General Public License, v2  |
|----------------------------------------------------------|
|  For documentation, citation & bug-report instructions:  |
|        http://pngu.mgh.harvard.edu/purcell/plink/        |
@----------------------------------------------------------@

Skipping web check... [ --noweb ] 
Writing this text to log file [ all_sitesBT0_burden.log ]
Analysis started: Tue Dec 19 14:33:51 2017

Options in effect:
	--map all.cnv.map
	--fam ../simplex.ped
	--cnv-list all.cnv
	--noweb
	--1
	--cnv-check-no-overlap
	--cnv-sites 0
	--out all_sitesBT0_burden

Reading marker information from [ all.cnv.map ]
3733 (of 3733) markers to be included from [ all.cnv.map ]
Reading individual information from [ ../simplex.ped ]
Reading pedigree information from [ ../simplex.ped

	--cnv-subset ../genes.txt
	--cnv-del
	--cnv-sites 0
	--out all_sitesBT0_delGenes

Reading marker information from [ all.cnv.map ]
3733 (of 3733) markers to be included from [ all.cnv.map ]
Reading individual information from [ ../simplex.ped ]
Reading pedigree information from [ ../simplex.ped ] 
99 individuals read from [ ../simplex.ped ] 
99 individuals with nonmissing phenotypes
Assuming a disease phenotype (0=unaff, 1=aff, other=miss)
21 cases, 78 controls and 0 missing
56 males, 43 females, and 0 of unspecified sex
Reading intersection subset list from [ ../genes.txt ]
Looking for subset of 25303 ranges
Reading CNV intersection list from [ ../glist-hg19 ]
Read 25646 ranges to intersect with CNV list

Reading segment list (CNVs) from [ all.cnv ]
Writing back list to intersected regions to [ all_sitesBT0_delGenes.reg ]
1882 mapped to a person, of which 1416 passed filters
1410 intersected with one or more specified region
1410 of 1882 mapped as valid segments
 CopyN Case/Control
  

21 cases, 78 controls and 0 missing
56 males, 43 females, and 0 of unspecified sex
Reading intersection subset list from [ ../hg19_allenBrainGene_trimmed.txt ]
Looking for subset of 606 ranges
Reading CNV intersection list from [ ../glist-hg19 ]
Read 577 ranges to intersect with CNV list
Could not find 29 ranges
Writing this list to [ all_sitesBT0_dupBrainGenes.notfound ] 

Reading segment list (CNVs) from [ all.cnv ]
Writing back list to intersected regions to [ all_sitesBT0_dupBrainGenes.reg ]
1882 mapped to a person, of which 466 passed filters
22 intersected with one or more specified region
22 of 1882 mapped as valid segments
 CopyN Case/Control
     3       7 / 15

Writing per-individual summary to [ all_sitesBT0_dupBrainGenes.cnv.indiv ]
Writing positional summary to [ all_sitesBT0_dupBrainGenes.cnv.summary ]

Analysis finished: Tue Dec 19 14:33:57 2017


@----------------------------------------------------------@
|        PLINK!       |     v1.07      |   10/Aug/2009     |
|--


Analysis finished: Tue Dec 19 14:33:59 2017


@----------------------------------------------------------@
|        PLINK!       |     v1.07      |   10/Aug/2009     |
|----------------------------------------------------------|
|  (C) 2009 Shaun Purcell, GNU General Public License, v2  |
|----------------------------------------------------------|
|  For documentation, citation & bug-report instructions:  |
|        http://pngu.mgh.harvard.edu/purcell/plink/        |
@----------------------------------------------------------@

Skipping web check... [ --noweb ] 
Writing this text to log file [ all_sitesBT5_delGenes.log ]
Analysis started: Tue Dec 19 14:33:59 2017

Options in effect:
	--map all.cnv.map
	--fam ../simplex.ped
	--cnv-list all.cnv
	--noweb
	--1
	--cnv-intersect ../glist-hg19
	--cnv-verbose-report-regions
	--cnv-subset ../genes.txt
	--cnv-del
	--cnv-sites 5
	--out all_sitesBT5_delGenes

Reading marker information from [ all.cnv.map ]
3733 (of 3733) markers to be included f


Skipping web check... [ --noweb ] 
Writing this text to log file [ all_sitesBT5_dupBrainGenes.log ]
Analysis started: Tue Dec 19 14:34:02 2017

Options in effect:
	--map all.cnv.map
	--fam ../simplex.ped
	--cnv-list all.cnv
	--noweb
	--1
	--cnv-intersect ../glist-hg19
	--cnv-verbose-report-regions
	--cnv-subset ../hg19_allenBrainGene_trimmed.txt
	--cnv-dup
	--cnv-sites 5
	--out all_sitesBT5_dupBrainGenes

Reading marker information from [ all.cnv.map ]
3733 (of 3733) markers to be included from [ all.cnv.map ]
Reading individual information from [ ../simplex.ped ]
Reading pedigree information from [ ../simplex.ped ] 
99 individuals read from [ ../simplex.ped ] 
99 individuals with nonmissing phenotypes
Assuming a disease phenotype (0=unaff, 1=aff, other=miss)
21 cases, 78 controls and 0 missing
56 males, 43 females, and 0 of unspecified sex
Reading intersection subset list from [ ../hg19_allenBrainGene_trimmed.txt ]
Looking for subset of 606 ranges
Reading CNV intersection list from [

...


Skipping web check... [ --noweb ] 
Writing this text to log file [ inherited_comb.5_clean_sitesBT10_burden.log ]
Analysis started: Tue Dec 19 14:36:29 2017

Options in effect:
	--map inherited_comb.5_clean.cnv.map
	--fam ../simplex.ped
	--cnv-list inherited_comb.5_clean.cnv
	--noweb
	--1
	--cnv-check-no-overlap
	--cnv-sites 10
	--out inherited_comb.5_clean_sitesBT10_burden

** For gPLINK compatibility, do not use '.' in --out **
Reading marker information from [ inherited_comb.5_clean.cnv.map ]
62 (of 62) markers to be included from [ inherited_comb.5_clean.cnv.map ]
Reading individual information from [ ../simplex.ped ]
Reading pedigree information from [ ../simplex.ped ] 
99 individuals read from [ ../simplex.ped ] 
99 individuals with nonmissing phenotypes
Assuming a disease phenotype (0=unaff, 1=aff, other=miss)
21 cases, 78 controls and 0 missing
56 males, 43 females, and 0 of unspecified sex

Reading segment list (CNVs) from [ inherited_comb.5_clean.cnv ]
No overlapping samples 


Skipping web check... [ --noweb ] 
Writing this text to log file [ inherited_comb.5_clean_sitesBT10_delGenes.log ]
Analysis started: Tue Dec 19 14:36:29 2017

Options in effect:
	--map inherited_comb.5_clean.cnv.map
	--fam ../simplex.ped
	--cnv-list inherited_comb.5_clean.cnv
	--noweb
	--1
	--cnv-intersect ../glist-hg19
	--cnv-verbose-report-regions
	--cnv-subset ../genes.txt
	--cnv-del
	--cnv-sites 10
	--out inherited_comb.5_clean_sitesBT10_delGenes

** For gPLINK compatibility, do not use '.' in --out **
Reading marker information from [ inherited_comb.5_clean.cnv.map ]
62 (of 62) markers to be included from [ inherited_comb.5_clean.cnv.map ]
Reading individual information from [ ../simplex.ped ]
Reading pedigree information from [ ../simplex.ped ] 
99 individuals read from [ ../simplex.ped ] 
99 individuals with nonmissing phenotypes
Assuming a disease phenotype (0=unaff, 1=aff, other=miss)
21 cases, 78 controls and 0 missing
56 males, 43 females, and 0 of unspecified sex
Reading i

|        PLINK!       |     v1.07      |   10/Aug/2009     |
|----------------------------------------------------------|
|  (C) 2009 Shaun Purcell, GNU General Public License, v2  |
|----------------------------------------------------------|
|  For documentation, citation & bug-report instructions:  |
|        http://pngu.mgh.harvard.edu/purcell/plink/        |
@----------------------------------------------------------@

Skipping web check... [ --noweb ] 
Writing this text to log file [ inherited_comb.5_clean_sitesBT10_dupBrainGenes.log ]
Analysis started: Tue Dec 19 14:36:30 2017

Options in effect:
	--map inherited_comb.5_clean.cnv.map
	--fam ../simplex.ped
	--cnv-list inherited_comb.5_clean.cnv
	--noweb
	--1
	--cnv-intersect ../glist-hg19
	--cnv-verbose-report-regions
	--cnv-subset ../hg19_allenBrainGene_trimmed.txt
	--cnv-dup
	--cnv-sites 10
	--out inherited_comb.5_clean_sitesBT10_dupBrainGenes

** For gPLINK compatibility, do not use '.' in --out **
Reading marker information f

Writing back list to intersected regions to [ inherited_comb.5_clean_sitesBT15_brainGenes.reg ]
35 mapped to a person, of which 3 passed filters
1 intersected with one or more specified region
1 of 35 mapped as valid segments
 CopyN Case/Control
     1        1 / 0

Writing per-individual summary to [ inherited_comb.5_clean_sitesBT15_brainGenes.cnv.indiv ]
Writing positional summary to [ inherited_comb.5_clean_sitesBT15_brainGenes.cnv.summary ]

Analysis finished: Tue Dec 19 14:36:30 2017


@----------------------------------------------------------@
|        PLINK!       |     v1.07      |   10/Aug/2009     |
|----------------------------------------------------------|
|  (C) 2009 Shaun Purcell, GNU General Public License, v2  |
|----------------------------------------------------------|
|  For documentation, citation & bug-report instructions:  |
|        http://pngu.mgh.harvard.edu/purcell/plink/        |
@----------------------------------------------------------@

Skipping web ch

21 cases, 78 controls and 0 missing
56 males, 43 females, and 0 of unspecified sex

Reading segment list (CNVs) from [ inherited_comb.5_clean.cnv ]
35 mapped to a person, of which 0 passed filters
0 of 35 mapped as valid segments
 CopyN Case/Control

Writing per-individual summary to [ inherited_comb.5_clean_sitesBT15_dupBurden.cnv.indiv ]
Writing positional summary to [ inherited_comb.5_clean_sitesBT15_dupBurden.cnv.summary ]

Analysis finished: Tue Dec 19 14:36:31 2017


@----------------------------------------------------------@
|        PLINK!       |     v1.07      |   10/Aug/2009     |
|----------------------------------------------------------|
|  (C) 2009 Shaun Purcell, GNU General Public License, v2  |
|----------------------------------------------------------|
|  For documentation, citation & bug-report instructions:  |
|        http://pngu.mgh.harvard.edu/purcell/plink/        |
@----------------------------------------------------------@

Skipping web check... [ --noweb ]